In [12]:
# Colab was used for this notebook hence the library installs
%%capture
!pip install pycaret --quiet --upgrade
!pip install dabl --quiet --upgrade
!pip install shap --quiet --upgrade
!pip install sdv  --quiet --upgrade
!pip install sdv[ctgan] --quiet --upgrade
!pip install baytune --quiet --upgrade
!pip install optuna --quiet --upgrade

UsageError: Line magic function `%%capture` not found.


In [9]:
pip freeze

alembic==1.4.1
argon2-cffi==20.1.0
async-generator==1.10
attrs==20.3.0
backcall==0.2.0
baytune==0.4.0
bleach==3.3.0
blis==0.7.4
Boruta==0.3
catalogue==1.0.0
certifi==2020.12.5
cffi==1.14.5
chardet==4.0.0
click==7.1.2
cliff==3.7.0
cloudpickle==1.6.0
cmaes==0.8.2
cmd2==1.5.0
colorama==0.4.4
colorlog==4.7.2
colorlover==0.3.0
confuse==1.4.0
copulas==0.5.0
ctgan==0.4.3
cufflinks==0.17.3
cycler==0.10.0
cymem==2.0.5
Cython==0.29.14
dabl==0.2.0
databricks-cli==0.14.2
dataclasses==0.8
decorator==4.4.2
deepecho==0.2.0
defusedxml==0.7.1
docker==4.4.4
entrypoints==0.3
Faker==4.14.2
Flask==1.1.2
funcy==1.15
future==0.18.2
gensim==3.8.3
gitdb==4.0.5
GitPython==3.1.14
graphviz==0.16
greenlet==1.0.0
htmlmin==0.1.12
idna==2.10
ImageHash==4.2.0
imbalanced-learn==0.7.0
importlib-metadata==3.7.3
ipykernel==5.5.0
ipython==7.16.1
ipython-genutils==0.2.0
ipywidgets==7.6.3
itsdangerous==1.1.0
jedi==0.18.0
Jinja2==2.11.3
joblib==1.0.1
jsonschema==3.2.0
jupyter-client==6.1.12
jupyter-core==4.7.1
jupyterlab-pygm

# Here we will present the bug of 1d-array

In [2]:
from pycaret.classification import * # Preprocessing, modelling, interpretation, deployment...
import pandas as pd # Basic data manipulation
#import dabl as db # Summary plot
from sklearn.model_selection import train_test_split # Data split
from sdv.tabular import CopulaGAN # Synthetic data
from sdv.evaluation import evaluate # Evaluate synthetic data
from btb.tuning import Tunable, GCPTuner # CopulaGAN optimising
from btb.tuning import hyperparams as hp  # Set hyperparameters for optimising
import joblib # Saving preparation steps

# *First we use fake data to present the bug*

In [3]:
x=np.random.rand(1000)*50
y=np.random.rand(1000)*50
z=np.random.rand(1000)*50
df=pd.DataFrame(pd.DataFrame([x,y,z]).values.T,columns = ["x","y","z"])

In [4]:
tuner = GCPTuner(Tunable({
          'epochs': hp.IntHyperParam(min = 80, max = 400),
          'batch_size' : hp.IntHyperParam(min = 1, max = 100),
          'embedding_dim' : hp.IntHyperParam(min = 1, max = 100),
          'gen' : hp.IntHyperParam(min = 1, max = 1000),
          'dim_gen' : hp.IntHyperParam(min = 1, max = 1000)
        }))

In [5]:
tracker = 0 
real = df
for _ in range(50):
    tracker += 1
    print(tracker)
    proposal = tuner.propose(1)
    model = CopulaGAN(embedding_dim = proposal['embedding_dim'],
                    generator_dim = (proposal['gen'], proposal['gen']),
                    discriminator_dim = (proposal['dim_gen'], proposal['dim_gen']),
                    batch_size = proposal['batch_size'] * 10,
                    epochs = proposal['epochs'])
    model.fit(real)
    synth_data = model.sample(600, max_retries = 300)
    score = evaluate(synthetic_data = synth_data, real_data = real)
    tuner.record(proposal, score)

1
2
3
4
5
6
7


ValueError: Expected 1d array, got [[0.5091826  0.76110796]
 [0.5091826  0.76110796]
 [0.5091826  0.76110796]
 ...
 [0.5091826  0.76110796]
 [0.5091826  0.76110796]
 [0.5091826  0.76110796]].

# *Now we present a real example.*

In [6]:
# Read and output the top 5 rows
hr_data = pd.read_csv("data/HR Employee Attrition.csv")
hr_data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


Try the data below several times and this error will appear.

In [7]:
tuner = GCPTuner(Tunable({
          'epochs': hp.IntHyperParam(min = 80, max = 400),
          'batch_size' : hp.IntHyperParam(min = 1, max = 100),
          'embedding_dim' : hp.IntHyperParam(min = 1, max = 100),
          'gen' : hp.IntHyperParam(min = 1, max = 1000),
          'dim_gen' : hp.IntHyperParam(min = 1, max = 1000)
        }))

In [8]:
best_score = 0 # Keep track of best score
tracker = 0 # Keep track of how many loops have completed

real = hr_data[hr_data["Attrition"] == "Yes"] # Filter to only those employees that left

## TRAINING LOOP START ##
for _ in range(50):

  # Increment the tracker
    tracker += 1

  # Every 5 loops output the tracker
    if tracker % 1 == 0:
        print(tracker)

  # Get the hyperparameters for this loop
    proposal = tuner.propose(1)
  
  # Create the CopulaGAN
  # NOTE - batch_size is multiplied by 10 as needs to be a factor of 10
    model = CopulaGAN(primary_key = "EmployeeNumber", 
                    embedding_dim = proposal['embedding_dim'],
                    generator_dim = (proposal['gen'], proposal['gen']),
                    discriminator_dim = (proposal['dim_gen'], proposal['dim_gen']),
                    batch_size = proposal['batch_size'] * 10,
                    epochs = proposal['epochs'])
  
  # Fit the CopulaGAN
    model.fit(real)
  
  # Create 600 rows of data
    synth_data = model.sample(600, max_retries = 300)
  
  # Evaluate the synthetic data against the real data
    score = evaluate(synthetic_data = synth_data, real_data = real)

  # If the new hyperparameters beat the best ones, store them along with the score
    if score > best_score:
        best_params = proposal
        best_score = score

  # Record the hyperparameters and score      
    tuner.record(proposal, score)

## TRAINING LOOP END ##


print('Best score obtained: ', best_score)
print('Best parameters: ', best_params)

1
2
3
4
5
6
7


ValueError: Expected 1d array, got [[0.50518258 0.76416991]
 [0.50518258 0.76416991]
 [0.50518258 0.76416991]
 ...
 [0.50518258 0.76416991]
 [0.50518258 0.76416991]
 [0.50518258 0.76416991]].